В данном упражнении вам предстоит решить уже знакомую задачу классификации изображений – отделить изображения кошек от изображений собак, используя ансамбль моделей на основе стекинга.

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
import cv2
from imutils import paths
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

In [ ]:
!jar xvf /content/train_task_.zip

  created: train/
 inflated: train/.DS_Store
 inflated: train/cat.0.jpg
 inflated: train/cat.1.jpg
 inflated: train/cat.10.jpg
 inflated: train/cat.100.jpg
 inflated: train/cat.101.jpg
 inflated: train/cat.102.jpg
 inflated: train/cat.103.jpg
 inflated: train/cat.104.jpg
 inflated: train/cat.105.jpg
 inflated: train/cat.106.jpg
 inflated: train/cat.107.jpg
 inflated: train/cat.108.jpg
 inflated: train/cat.109.jpg
 inflated: train/cat.11.jpg
 inflated: train/cat.110.jpg
 inflated: train/cat.111.jpg
 inflated: train/cat.112.jpg
 inflated: train/cat.113.jpg
 inflated: train/cat.114.jpg
 inflated: train/cat.115.jpg
 inflated: train/cat.116.jpg
 inflated: train/cat.117.jpg
 inflated: train/cat.118.jpg
 inflated: train/cat.119.jpg
 inflated: train/cat.12.jpg
 inflated: train/cat.120.jpg
 inflated: train/cat.121.jpg
 inflated: train/cat.122.jpg
 inflated: train/cat.123.jpg
 inflated: train/cat.124.jpg
 inflated: train/cat.125.jpg
 inflated: train/cat.126.jpg
 inflated: train/cat.127.jpg
 infl

In [ ]:
!jar xvf /content/test_task_.zip

  created: test/
 inflated: test/.DS_Store
  created: test/.ipynb_checkpoints/
 inflated: test/cat.1000.jpg
 inflated: test/cat.1001.jpg
 inflated: test/cat.1002.jpg
 inflated: test/cat.1003.jpg
 inflated: test/cat.1004.jpg
 inflated: test/cat.1005.jpg
 inflated: test/cat.1006.jpg
 inflated: test/cat.1007.jpg
 inflated: test/cat.1008.jpg
 inflated: test/cat.1009.jpg
 inflated: test/cat.1010.jpg
 inflated: test/cat.1011.jpg
 inflated: test/cat.1012.jpg
 inflated: test/cat.1013.jpg
 inflated: test/cat.1014.jpg
 inflated: test/cat.1015.jpg
 inflated: test/cat.1016.jpg
 inflated: test/cat.1017.jpg
 inflated: test/cat.1018.jpg
 inflated: test/cat.1019.jpg
 inflated: test/cat.1020.jpg
 inflated: test/cat.1021.jpg
 inflated: test/cat.1022.jpg
 inflated: test/cat.1023.jpg
 inflated: test/cat.1024.jpg
 inflated: test/cat.1025.jpg
 inflated: test/cat.1026.jpg
 inflated: test/cat.1027.jpg
 inflated: test/cat.1028.jpg
 inflated: test/cat.1029.jpg
 inflated: test/cat.1030.jpg
 inflated: test/cat.10

In [ ]:
def extract_histogram(image, bins=(8, 8, 8)):
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

In [ ]:
X_train = []
y_train = []
imagePaths = sorted(list(paths.list_images('train')))
for image in imagePaths:
  image1 = cv2.imread(image)
  X_train.append(extract_histogram(image1))
  y_train.append(1) if 'cat' in image else y_train.append(0)

In [ ]:
X_test = []
y_test = []
imagePaths = sorted(list(paths.list_images('test')))
for image in imagePaths:
    image1 = cv2.imread(image)
    X_test.append(extract_histogram(image1))
    y_test.append(1) if 'cat' in image else y_test.append(0)

In [ ]:
linearSVC = LinearSVC(C = 1.25, random_state=80).fit(X_train, y_train)

In [ ]:
decisionTreeClassifier = DecisionTreeClassifier(criterion = 'entropy', min_samples_leaf = 10, max_leaf_nodes = 20, random_state = 80).fit(X_train, y_train)

In [ ]:
baggingClassifier = BaggingClassifier(base_estimator=decisionTreeClassifier, n_estimators = 13, random_state = 80).fit(X_train, y_train)

In [ ]:
randomForestClassifier = RandomForestClassifier(n_estimators = 13, criterion = 'entropy', min_samples_leaf = 10, max_leaf_nodes = 20, random_state = 80).fit(X_train, y_train)

In [ ]:
logisticRegression = LogisticRegression(solver='lbfgs', random_state = 80).fit(X_train, y_train)

In [ ]:
estimators = [('svc', linearSVC), ('bc', baggingClassifier), ('rfc', randomForestClassifier)]

In [ ]:
clf = StackingClassifier(
     estimators=estimators, final_estimator=logisticRegression, cv = 2
 ).fit(X_train, y_train)

clf.score(X_train, y_train)

0.843

In [ ]:
X_control = []
image1 = cv2.imread('/content/test/dog.1011.jpg')
X_control.append(extract_histogram(image1))
clf.predict_proba(X_control)

array([[0.58823011, 0.41176989]])